<a href="https://colab.research.google.com/github/bacoco/LLM_train/blob/main/Mamba_Hermes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q torch==2.1.0 transformers==4.35.0 causal-conv1d==1.0.0 mamba-ssm==1.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.7 MB/s eta 0:00:00


In [2]:
#fix AssertionError: libcuda.so cannot found!
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig


/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link



In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

CHAT_TEMPLATE_ID = "HuggingFaceH4/zephyr-7b-beta"

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "clibrain/mamba-2.8b-instruct-openhermes"

eos_token = "<|endoftext|>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.eos_token = eos_token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(CHAT_TEMPLATE_ID).chat_template

model = MambaLMHeadModel.from_pretrained(
        model_name, device=device, dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

messages = []
prompt = "Tell me 5 sites to visit in Bangalore"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=2000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

messages.append(dict(role="assistant", content=assistant_message))


print(assistant_message)


1. Mysore Palace: This is a royal palace located in Mysore, Karnataka, India. It is one of the most popular tourist attractions in the city.

2. St. Mary’s Church: This is a historic church located in the heart of Bangalore. It is a popular tourist spot and is known for its beautiful architecture.

3. Cubbon Park: This is a large public park located in the heart of Bangalore. It is a popular spot for people to relax, walk, and enjoy the beautiful weather.

4. Vidhana Soudha: This is the administrative headquarters of the Karnataka state government. It is a beautiful building with a lot of historical significance.

5. Lalbagh Fort: This is a fort located in the heart of Bangalore. It is a popular tourist spot and is known for its beautiful gardens and architecture.


In [5]:
prompt = "maybe in Chennai?"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=2000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

messages.append(dict(role="assistant", content=assistant_message))


print(assistant_message)

1. Marina Beach: This is a beautiful beach located in Chennai, Tamil Nadu, India. It is a popular tourist spot and is known for its clear waters, beautiful sunsets, and peaceful atmosphere.

2. Fort St. George: This is a historic fort located in Chennai, Tamil Nadu, India. It is a popular tourist spot and is known for its beautiful architecture and history.

3. Anna Square: This is a large public square located in Chennai, Tamil Nadu, India. It is a popular spot for people to relax, walk, and enjoy the beautiful weather.

4. Chinna Thambireddige Temple: This is a Hindu temple located in Chennai, Tamil Nadu, India. It is a popular tourist spot and is known for its beautiful architecture and peaceful atmosphere.

5. Chennai Central: This is a railway station located in Chennai, Tamil Nadu, India. It is a popular tourist spot and is known for its beautiful architecture and history.


In [6]:
messages

[{'role': 'user', 'content': 'Tell me 5 sites to visit in Bangalore'},
 {'role': 'assistant',
  'content': '1. Mysore Palace: This is a royal palace located in Mysore, Karnataka, India. It is one of the most popular tourist attractions in the city.\n\n2. St. Mary’s Church: This is a historic church located in the heart of Bangalore. It is a popular tourist spot and is known for its beautiful architecture.\n\n3. Cubbon Park: This is a large public park located in the heart of Bangalore. It is a popular spot for people to relax, walk, and enjoy the beautiful weather.\n\n4. Vidhana Soudha: This is the administrative headquarters of the Karnataka state government. It is a beautiful building with a lot of historical significance.\n\n5. Lalbagh Fort: This is a fort located in the heart of Bangalore. It is a popular tourist spot and is known for its beautiful gardens and architecture.'},
 {'role': 'user', 'content': 'maybe in Chennai?'},
 {'role': 'assistant',
  'content': '1. Marina Beach: T

In [7]:
prompt = "how about Germany?"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=2000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

messages.append(dict(role="assistant", content=assistant_message))


print(assistant_message)

1. Berlin Wall Memorial: This is a memorial located in Berlin, Germany, that commemorates the fall of the Berlin Wall. It is a popular tourist spot and is known for its beautiful architecture and history.

2. Brandenburg Gate: This is a gate located in Berlin, Germany, that was once the main entrance to the city. It is a popular tourist spot and is known for its beautiful architecture and history.

3. Pergamon Museum: This is a museum located in Berlin, Germany, that is dedicated to the history and culture of ancient Persia and the Middle East. It is a popular tourist spot and is known for its beautiful architecture and history.

4. Berlin Wall Memorial: This is a memorial located in Berlin, Germany, that commemorates the fall of the Berlin Wall. It is a popular tourist spot and is known for its beautiful architecture and history.

5. Brandenburg Gate: This is a gate located in Berlin, Germany, that was once the main entrance to the city. It is a popular tourist spot and is known for i

In [8]:

messages = []
prompt = "Tell me 5 sites to visit in Bangalore in 1 day and give me the response in json"
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=2000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

messages.append(dict(role="assistant", content=assistant_message))


print(assistant_message)


Sure, here are 5 sites to visit in Bangalore in 1 day:
1. Mysore Palace
2. St. Mary's Cathedral
3. Cubbon Park
4. Lalbagh Fort
5. Vidhana Soudha
Here is the response in JSON format:
```
{
  "sites": [
    {
      "name": "Mysore Palace",
      "address": "Mysore Road, Mysore",
      "opening_hours": "9:00 AM - 6:00 PM",
      "description": "Mysore Palace is a royal palace located in Mysore, Karnataka, India. It was built in the 17th century and is one of the most popular tourist attractions in Mysore."
    },
    {
      "name": "St. Mary's Cathedral",
      "address": "St. Mary's Road, Bangalore",
      "opening_hours": "9:00 AM - 5:00 PM",
      "description": "St. Mary's Cathedral is a historic church located in Bangalore, Karnataka, India. It was built in the Gothic Revival style and is one of the oldest churches in Bangalore."
    },
    {
      "name": "Cubbon Park",
      "address": "Cubbon Road, Bangalore",
      "opening_hours": "7:00 AM - 7:00 PM",
      "description": "Cubb

In [22]:
text="""Napoleon Bonaparte (born Napoleone di Buonaparte;[1][b] 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He initiated many enduring reforms, but has been criticized for his authoritarian rule. He is considered one of the greatest military commanders in history and his wars and campaigns are still studied at military schools worldwide. However, historians still debate whether he was responsible for the Napoleonic Wars in which between three and six million people died.[2][3]

Napoleon was born on the island of Corsica into a family descended from Italian nobility.[4][5] He was resentful of the French monarchy, and supported the French Revolution in 1789 while serving in the French army, trying to spread its ideals to his native Corsica. He rose rapidly in the ranks after saving the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories, and became a national hero. Two years later he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. In 1804, to consolidate and expand his power, he crowned himself Emperor of the French.

Differences with the United Kingdom meant France faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm campaign and at the Battle of Austerlitz, which led to the dissolution of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him. Napoleon defeated Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, and defeated the Russians in June 1807 at Friedland, forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram.

Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph the King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War aided by a British army, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France, resulting in a large coalition army defeating Napoleon at the Battle of Leipzig. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power.

Napoleon escaped in February 1815 and took control of France.[6] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51.

Napoleon had a lasting impact on the world, bringing modernizing reforms to France and Western Europe[c] and stimulating the development of nation states. He also sold the Louisiana Territory to the United States in 1803, doubling the latter's size.[2][13] However, his mixed record on civil rights and exploitation of conquered territories adversely affect his reputation.[d]

Early life
Half-length portrait of a wigged middle-aged man with a well-to-do jacket. His left hand is tucked inside his waistcoat.
Napoleon's father, Carlo Buonaparte, fought for Corsican independence under Pasquale Paoli, but after their defeat he eventually became the island's representative to the court of Louis XVI.
Napoleon's family was of Italian origin. His paternal ancestors, the Buonapartes, descended from a minor Tuscan noble family that emigrated to Corsica in the 16th century and his maternal ancestors, the Ramolinos, descended from a minor Genoese noble family.[18] His parents Carlo Maria Buonaparte and Maria Letizia Ramolino maintained a home in Ajaccio where Napoleon was born on 15 August 1769. He was the family's fourth child and third son.[e] He had an elder brother, Joseph, and younger siblings Lucien, Elisa, Louis, Pauline, Caroline, and Jérôme. Napoleon was baptized as a Catholic, under the name Napoleone.[19] In his youth, his name was also spelled as Nabulione, Nabulio, Napolionne, and Napulione.[20]

Napoleon was born one year after the Republic of Genoa ceded Corsica to France.[21] The state sold sovereign rights a year before his birth and the island was conquered by France during the year of his birth. It was formally incorporated as a province in 1770, after 500 years under Genoese rule and 14 years of independence.[f] Napoleon's parents joined the Corsican resistance and fought against the French to maintain independence, even when Maria was pregnant with him. His father Carlo was an attorney who had supported and actively collaborated with patriot Pasquale Paoli during the Corsican war of independence against France;[5] after the Corsican defeat at Ponte Novu in 1769 and Paoli's exile in Britain, Carlo began working for the new French government and in 1777 was named representative of the island to the court of Louis XVI.[5][25]

Half-length portrait of a middle-aged woman with bonnet and expensive burgundy shawl
Madame Mère, painted by Joseph Karl Stieler (1811)
The dominant influence of Napoleon's childhood was his mother, whose firm discipline restrained a rambunctious child.[25] Later in life, Napoleon said, "The future destiny of the child is always the work of the mother."[26] His maternal grandmother had married into the Swiss Fesch family in her second marriage, and Napoleon's uncle, the cardinal Joseph Fesch, fulfilled a role as protector of the Bonaparte family for some years. Napoleon's noble, moderately affluent background afforded him greater opportunities to study than were available to a typical Corsican of the time.[27]


Statue of Bonaparte as a schoolboy in Brienne, aged 15, by Louis Rochet [fr] (1853)
When he turned 9 years old, Napoleon moved to the French mainland and enrolled at a religious school in Autun in January 1779. In May, he transferred with a scholarship to a military academy at Brienne-le-Château.[28] In his youth, he was an outspoken Corsican nationalist and supported the state's independence from France.[29][30] Like many Corsicans, Napoleon spoke and read Corsican (as his mother tongue) and Italian (as the official language of Corsica).[31][32][33][30] He began learning French in school at the age of around 10.[34] Although he became fluent in French, he spoke with a distinctive Corsican accent and never learned to spell in French.[35] Consequently, Napoleon was routinely bullied by his peers for his accent, birthplace, short stature, mannerisms, and inability to speak French quickly.[32] He became reserved and melancholic, applying himself to reading. An examiner observed that Napoleon "has always been distinguished for his application in mathematics. He is fairly well acquainted with history and geography ... This boy would make an excellent sailor".[g][37]

One story told of Napoleon at the school is that he led junior students to victory against senior students in a snowball fight, showing his leadership abilities.[38] In early adulthood, Napoleon briefly intended to become a writer; he authored a history of Corsica and a romantic novella.[29]

On completion of his studies at Brienne in 1784, Napoleon was admitted to the École militaire in Paris. He trained to become an artillery officer and, when his father's death reduced his income, was forced to complete the two-year course in one year.[39] He was the first Corsican to graduate from the École militaire.[39] He was examined by the famed scientist Pierre-Simon Laplace.[40]

Early career

Bonaparte, aged 23, as lieutenant-colonel of a battalion of Corsican Republican volunteers. Portrait by Henri Félix Emmanuel Philippoteaux
Upon graduating in September 1785, Bonaparte was commissioned a second lieutenant in La Fère artillery regiment.[h][28] He served in Valence and Auxonne until after the outbreak of the French Revolution in 1789. Bonaparte was a fervent Corsican nationalist during this period.[42] He asked for leave to join his mentor Paoli, when Paoli was allowed to return to Corsica by the National Assembly. But Paoli had no sympathy for Napoleon, as he deemed his father a traitor for having deserted the cause of Corsican independence.[43]

He spent the early years of the Revolution in Corsica, fighting in a complex three-way struggle among royalists, revolutionaries, and Corsican nationalists. Napoleon embraced the ideals of the Revolution, becoming a supporter of the Jacobins and joining the pro-French Corsican Republicans who opposed Paoli's policy and his aspirations to secede.[44] He was given command over a battalion of volunteers and promoted to captain in the regular army in 1792, despite exceeding his leave of absence and leading a riot against French troops.[45]

When Corsica declared formal secession from France and requested the protection of the British government, Napoleon and his commitment to the French Revolution came into conflict with Paoli, who had decided to sabotage the Corsican contribution to the Expédition de Sardaigne by preventing a French assault on the Sardinian island La Maddalena.[46] Bonaparte and his family were compelled to flee to Toulon on the French mainland in June 1793 because of the split with Paoli.[28]

Although he was born "Napoleone Buonaparte", it was after this that Napoleon began styling himself "Napoléon Bonaparte". His family did not drop the name Buonaparte until 1796. The first known record of him signing his name as Bonaparte was at the age of 27 (in 1796).[47][19][48]

Siege of Toulon
Main article: Siege of Toulon (1793)

Bonaparte at the Siege of Toulon, 1793, by Edouard Detaille
In July 1793, Bonaparte published a pro-republican pamphlet, Le souper de Beaucaire (Supper at Beaucaire), which gained him the support of Augustin Robespierre, the younger brother of the Revolutionary leader Maximilien Robespierre. With the help of his fellow Corsican Antoine Christophe Saliceti, Bonaparte was appointed senior gunner and artillery commander of the republican forces that arrived at Toulon on 8 September.[49][50]

He adopted a plan to capture a hill where republican guns could dominate the city's harbour and force the British to evacuate. The assault on the position led to the capture of the city, and during it Bonaparte was wounded in the thigh on 16 December. Catching the attention of the Committee of Public Safety, he was put in charge of the artillery of France's Army of Italy.[51] On 22 December he was on his way to a new post in Nice, promoted from colonel to brigadier general at the age of 24. He devised plans to attack the Kingdom of Sardinia as part of France's campaign against the First Coalition.

The French army carried out Bonaparte's plan in the Battle of Saorgio in April 1794, and then advanced to seize Ormea in the mountains. From Ormea, it headed west to outflank the Austro-Sardinian positions around Saorge. After this campaign, Augustin Robespierre sent Bonaparte on a mission to the Republic of Genoa to determine the country's intentions towards France.[52]

"""

In [ ]:
messages = []
prompt = "give me a summary in bullet points format of followong text: "+text
messages.append(dict(role="user", content=prompt))

input_ids = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
).to(device)

out = model.generate(
    input_ids=input_ids,
    max_length=20000,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

decoded = tokenizer.batch_decode(out)
assistant_message = (
    decoded[0].split("<|assistant|>\n")[-1].replace(eos_token, "")
)

messages.append(dict(role="assistant", content=assistant_message))


print(assistant_message)